# SC1015 Mini Project: Signature Forgery Detection

  Welcome to our mini project! This first section will run you through the overview of what our project is about. We decided to follow the Datapipeline taught to us in the DataScience portion of this module: 
  
  1. __Practical Motivation:__ Why do we choose this problem?
  2. __Sample Collection:__ Which dataset do we use?
  3. __Problem Formulation:__ Rephrasing the problem into a datascience problem
  4. __Data Preparation:__ Steps needed to setup the dataset
  5. __Statistical Description:__ Statistics to describe our input data
  6. __Exploratory Analysis:__ What can we analyse from the statistics of our data?
  7. __Pattern Recognition:__ What patterns/features can we find from our data?
  8. __Analytic Visualization:__ How can we visualise what we're working with?
  9. __Machine Learning:__ Which model do we use to solve our problem?
  10. __Algorithmic Optimization:__ How can we optimise our algorithm? 
  11. __Statistical Inference:__ What can we infer from the results?
  12. __Information Presentation:__ Side by side comparison of the results

## Imports
  The section below will import all the necessary libraries for this project. Although, there are some pre-requisites needed to install into your python CMD or anaconda CMD. 
  
  * __pip install "tensorflow<2.11"__
  * __pip install opencv-python__

In [1]:
# Import relevant libraries
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras
import cv2 
from PIL import Image

ModuleNotFoundError: No module named 'tensorflow'

## Practical Motivation
  Signature forgery is a problem that bypasses an untrained eye. Not everyone has the patience to meticulously check the authenticity of thousands of documents. In fact, a Singaporean man was able to forge documents that helped him get 38 jobs in 4 years ([CNA](https://www.channelnewsasia.com/singapore/man-forge-documents-nus-degree-get-jobs-38-companies-890881), 2019). 
  ![Signature Forgery](https://www.martypearce.com/wp-content/uploads/2019/01/3e406f_00f3feb43e17449c8bfacc0f850bf362mv2.jpg)
  Forgery is a real concern and humans are bound to make mistakes when checking for forged documents. On a tiring day, we may not always be able to find forged documents and so our group strives to build a working model that can consistently detect Signature Forgery.

## Sample Collection
  The dataset we will be using is from Kaggle: [https://www.kaggle.com/datasets/robinreni/signature-verification-dataset](https://www.kaggle.com/datasets/robinreni/signature-verification-dataset). 
  It has the following directory:
* sign_data
  * test
  * train
  * test_data.csv
  * train_data.csv
  
  
1. There are a total of 2,149 images(png)
2. A training set of 69 different individuals' signatures, separated into a genuine and forged set of images
3. A test set of 21 different individuals' signatures, separated into a genuine and forged set of images

## Problem Formulation
  In terms of Data Science, this is definitely a "Classification" type of problem and we will be working with unstructured data, namely images. If we were to phrase this problem we would ask:
  
  __Is this image a genuine or forged signature?__
  
  This is a binary type of classification

## Data Preparation

Before we start with the analyses of the dataset, we first restructure the folder layout of the dataset.

### **Before** 
```
Root_Folder
└── Dataset/
    ├── test/
    │   ├── 049/
    │   │   ├── 01_049.png
    │   │   ├── 02_049.png
    │   │   └── ...
    │   ├── 049_forg/
    │   │   ├── 01_0114049.png
    │   │   ├── 01_0206049.png
    │   │   └── ...
    │   ├── 050/
    │   ├── 050_forg/
    │   └── ...
    ├── train/
    │   ├── 001/
    │   │   ├── 001_01.png
    │   │   ├── 001_02.png
    │   │   └── ...
    │   ├── 001_forg/
    │   │   ├── 0119001_01.png
    │   │   ├── 0119001_02.png
    │   │   └── ...
    │   ├── 002/
    │   ├── 002_forg/
    │   └── ...
    ├── test_data.csv
    └── train_data.csv
```

### **After**
```
Root_Folder
└── Dataset/
    ├── test/
    │   ├── forged/
    │   │   ├── 049_forg/
    │   │   │   ├── 049_forg_01.png
    │   │   │   ├── 049_forg_02.png
    │   │   │   └── ...
    │   │   ├── 050_forg/
    │   │   ├── 051_forg/
    │   │   └── ...
    │   └── real/
    │       ├── 049/
    │       │   ├── 01_049.png
    │       │   ├── 02_049.png
    │       │   └── ...
    │       ├── 050/
    │       ├── 051/
    │       └── ...
    ├── train/
    │   ├── forged/
    │   │   ├── 001_forg/
    │   │   │   ├── 001_forg_01.png
    │   │   │   ├── 001_forg_02.png
    │   │   │   └── ...
    │   │   ├── 002_forg/
    │   │   └── ...
    │   └── real/
    │       ├── 001/
    │       │   ├── 001_01.png
    │       │   ├── 001_02.png
    │       │   └── ...
    │       ├── 002/
    │       └── ...
    ├── test_data.csv
    └── train_data.csv
```


We added 2 folders(real & forged) in each **train** and **test** folders to categorise the signatures for better access.

Next, we moved on to actual preparation of the data. As the data is already split into **train** and **test** set, we do not need further splitting of the data.

Additionally, no data cleaning is required as the images does not contain any noises(images are high definition of different signatures). Also, all the signatures have the been categorised based on the individuals, no further classifying the images is needed.

However, one thing we need to do is to convert the image into numerical value to do processing on the data extracted.

We have come up with 2 ways to convert the images into numerical values

    1) Converting images into 2D array of RGB values

    2) Converting images into different properties (will be covered more in depth later)

These 2 approaches will be used in the different solutions to address our problem.

### Image Conversion Method 1 - Converting images into 2D array of RGB Values. 

This is done using existing library (`cv2`) to read the image (`imread`) and convert them into RGB values

A detailed example is shown below

```python
img = cv2.imread(*filename*)
"""
Example value of img:

[[250 250 250 ... 250 250 250]
 [250 250 250 ... 250 250 250]
 [250 250 250 ... 250 250 250]
 ...
 [250 250 250 ... 250 250 250]
 [250 250 250 ... 250 250 250]
 [250 250 250 ... 250 250 250]]
"""
```

However, as our images are of different dimensions, we would need to resize/scale the images so that there are of the same dimensions. An example of the scaling function is shown below.

```python
"""
imageA: RGB values of first image
imageB: RGB values of second image
return: RGB values of both image A and image B

The function check for the smaller dimension 
of the images and scale up the smaller one 
to the dimension of the bigger one
"""
def image_resize(imageA, imageB):
    if not (imageA.shape == imageB.shape):
        # Scale the smaller dimension pictures
        if (imageA.shape < imageB.shape):
            dim = (imageB.shape[1], imageB.shape[0])
            imageA = cv2.resize(imageA, dim)
        elif (imageB.shape < imageA.shape):
            dim = (imageA.shape[1], imageA.shape[0])
            imageB = cv2.resize(imageB, dim)
        
    return imageA, imageB
```

## Statistical Description

### Feature extraction - Converting the preprocessed images into numbers and statistics that can be inputed into our CNN model. This data invokes the concept of clustering to classify data according to certain patterns. In this case, the patterns are the distinct features of a signature, namely:

##### ratio: The ratio of the width to height of the bounding box around the signature.
##### cent_y: The y-coordinate of the center of mass of the signature.
##### cent_x: The x-coordinate of the center of mass of the signature.
##### eccentricity: A measure of how elongated the signature is.
##### solidity: The ratio of the area of the signature to the area of the convex hull surrounding it.
##### skew_x: A measure of how asymmetric the signature is with respect to the x-axis.
##### skew_y: A measure of how asymmetric the signature is with respect to the y-axis.
##### kurt_x: A measure of how heavy the tails of the signature are with respect to the x-axis.
##### kurt_y: A measure of how heavy the tails of the signature are with respect to the y-axis.
##### The outputs '0' and '1' indicate whether the signature is genuine or forged, with '1' representing a genuine signature and '0' representing a forged signature.

### For our specific case, instead of classifying data into specific categories, we use numeric metrics to quantify how much of that particular feature that signature possesses. 

In [ ]:
def Ratio(img):
    a = 0
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                a = a+1
    total = img.shape[0] * img.shape[1]
    return a/total

In [ ]:
def Centroid(img):
    numOfWhites = 0
    a = np.array([0,0])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                b = np.array([row,col])
                a = np.add(a,b)
                numOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/numOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]   # 1st row is the x-component, 2nd row is the y-component

In [ ]:
def EccentricitySolidity(img):
    r = regionprops(img.astype("int8"))
    return r[0].eccentricity, r[0].solidity

In [ ]:
def SkewKurtosis(img):
    h,w = img.shape
    x = range(w)  # columns value
    y = range(h)  # rows value
    #calculate projections along the x and y axes
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    #centroid
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    #standard deviation
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    
    #skewness
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)

    #Kurtosis
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    # 3 is subtracted to calculate relative to the normal distribution
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)

#### getFeatures function extracts the numerical values of the various features, and returns the data in the form of a tuple

#### getCSVFeatures uses this tuple and converts it into a 2-dimensional matrix as input format for CNN



In [ ]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = preproc(path, display=display)
    ratio = Ratio(img)
    centroid = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewness, kurtosis = SkewKurtosis(img)
    retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    return retVal

In [ ]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    temp = getFeatures(path, display=display)
    features = (temp[0], temp[1][0], temp[1][1], temp[2], temp[3], temp[4][0], temp[4][1], temp[5][0], temp[5][1])  
    '''
    2D MATRIX, DATA FORMAT FOR CNN INPUT. Each row vector displays each feature variable. 
    For example:
    
            col 1:         col 2:
    row 0: [ratio               ]
    row 1: [cent_y        cent_x]
    row 2: [eccentricity        ]
    row 3: [solidity            ]
    row 4: [skew_y        skew_x]
    row 5: [kurt_x        kurt_y]
    ''' 
    return features

### Saving the features

##### Saves the numerical variables of the signatures into an Excel CSV file

In [ ]:
def makeCSV(setType):
    # if the specified folder does not exist, create it to store the incoming CSV
    if not(os.path.exists('../Dataset/Features')):          
        os.mkdir('../Dataset/Features')   
        print('New folder "Features" created')
    if not(os.path.exists('../Dataset/Features/Training')):
        os.mkdir('../Dataset/Features/Training')
        print('New folder "Features/Training" created')
    if not(os.path.exists('../Dataset/Features/Testing')):
        os.mkdir('../Dataset/Features/Testing')
        print('New folder "Features/Testing" created')
   
    # Check what type
    if(setType != 'train' and setType != 'test'):
        print('Invalid type!')
        return
    
    if(setType == 'train'):
        gpath = genuine_image_paths_train
        fpath = forged_image_paths_train
        csvpath = '../Dataset/Features/Training/training_'
        setRange = range(1, 70)
    else:
        gpath = genuine_image_paths_test
        fpath = forged_image_paths_test
        csvpath = '../Dataset/Features/Testing/testing_'
        setRange = range(49, 70)
    

    # Extracting features based on type defined: 'train' ranges from 1-69, 'test' ranges from 49-69
    for person in tqdm(setRange, desc='Extracting Features...'):
        per = ('00'+str(person))[-3:]
               
        # In case folder is not iterative (i.e, 005 does not exist)
        if not(os.path.exists(os.path.join(gpath, per))):
            continue
    
        # In case folder is not iterative (i.e, 005 does not exist)
        if not(os.path.exists(os.path.join(fpath, per+'_forg'))):
            continue
    
        # every signature image will generate a unique CSV feature row specific to it, hence the concatenation
        with open(csvpath+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')

            # Genuine set
            i = 1
            fileNum = ('0' + str(i))[-2:]  # this takes care of single vs double digits
            filePath = os.path.join(gpath, per,  per + '_' + fileNum + '.png')
            while(os.path.exists(filePath)):
                source = filePath   # source image taken iteratively based on their own naming convention  
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')  # 1 is set for genuine signatures 
                i += 1
                fileNum = ('0' + str(i))[-2:]  # this takes care of single vs double digits
                filePath = os.path.join(gpath, per,  per + '_' + fileNum + '.png')
                
            # Forged set
            i = 1
            fileNum = ('0' + str(i))[-2:]  # this takes care of single vs double digits
            filePath = os.path.join(fpath, per+'_forg',  per + '_forg_' + fileNum + '.png')
            while(os.path.exists(filePath)):
                source = filePath   # source image taken iteratively based on their own naming convention  
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')  # 0 is set for genuine signatures 
                i += 1
                fileNum = ('0' + str(i))[-2:]  # this takes care of single vs double digits
                filePath = os.path.join(fpath, per+'_forg',  per + '_forg_' + fileNum + '.png')   

In [ ]:
makeCSV('train')
makeCSV('test')

New folder "Features" created
New folder "Features/Training" created
New folder "Features/Testing" created


Extracting Features...: 100%|██████████████████████████████████████████████████████████| 21/21 [20:17<00:00, 57.96s/it]


## Exploratory Analysis

## Pattern Recognition

## Analytic Visualization

## Machine Learning

## Algorithmic Optimization

## Statistical Inference

## Information Presentation